In [1]:
import re
from io import StringIO
import pandas as pd

In [2]:
def parse_x1(num, header=['Center Number', 'Atomic Number', 'Atomic Type', 'X', 'Y', 'Z']):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    so = re.finditer('(S|s)tandard orientation', outtxt)
    for s in so:
        pass
    last_so = s.end()
    delim = re.finditer('---------------------------------------------------------------------', outtxt[last_so:])
    
    i = 0
    for d in delim:
        if i == 1:
            first_delim_end = d.end()
        if i == 2:
            second_delim_start = d.start()
        i += 1
        
    return pd.read_csv(StringIO(outtxt[last_so+first_delim_end:last_so+second_delim_start]), delim_whitespace=True, header=None, names=header)
    

In [3]:
def parse_x2_occ(num):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    alpha_occ = re.finditer('(A|a)lpha  occ. eigenvalues --', outtxt)

    all_rows = []
    rows = []
    for occ in alpha_occ:
        l = outtxt[occ.end():occ.end()+51]
        match = re.search('A', l)
        if match is not None:
            print(match)
            rows.append(list(filter(lambda a: a != '', re.split('\s', outtxt[occ.end():occ.end()+match.start()-2]))))
            all_rows.append(rows)
            rows = []
        else:
            rows.append(list(filter(lambda a: a != '', re.split('\s', l))))
            
    return pd.DataFrame(all_rows[len(all_rows)-1])

In [4]:
def parse_x2_virt(num):
    out = open("../data/raw/outs/" + str(num) + ".out", "r")
    outtxt = out.read()
    alpha_occ = re.finditer('(A|a)lpha virt. eigenvalues --', outtxt)

    all_rows = []
    rows = []
    le = 0
    occ = next(alpha_occ, None)
    while occ is not None:
        le = occ.end()
        l = outtxt[occ.end():occ.end()+51]
        occ = next(alpha_occ, None)
        if occ is None or occ.start() - le > 55:
            rows.append(list(filter(lambda a: a != '' and a != 'Condensed' and a != 'to', re.split('\s', l))))
            all_rows.append(rows)
            rows = []
        else:
            rows.append(list(filter(lambda a: a != '', re.split('\s', l))))
            
    return pd.DataFrame(all_rows[len(all_rows)-1])

In [5]:
num = 186
parse_x2_virt(num)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/outs/186.out'